In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

**question 1:** What's the version.build_hash value?

In [3]:
e_search = Elasticsearch("http://localhost:9200")
e_search.info()["version"]["build_hash"] #


'42f05b9372a9a4a470db3b52817899b99a76ee73'

**Question 2:** Which function do you use for adding your data to elastic?

insert

index

put

add

In [4]:
Question_2 = "index"

### indexing the document

In [7]:
index = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

e_search.indices.create(index=index_name, body=index)

for doc in tqdm(documents):
    e_search.index(index=index_name, document=doc)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/1pJz-AUZR8OlTvUW27GpRw] already exists')

### implement elastic search

In [5]:
De_q = "How do I execute a command in a running docker container?"
De_course = "data-engineering-zoomcamp"
De_size = 5

In [14]:
search_query = {
    "size": De_size, #size of search output 
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": De_q, #query
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": De_course #filter
                }
            }
        }
    }
}

De_response = e_search.search(index=index_name, body=search_query)
for res in De_response["hits"]["hits"]:
    print(f"Section: {res['_source']['section']}")
    print(f"Question: {res['_source']['question']}")
    print(f"Answer: {res['_source']['text'][:60]}...\n")


Section: Module 1: Docker and Terraform
Question: PGCLI - running in a Docker container
Answer: In case running pgcli  locally causes issues or you do not w...

Section: Module 1: Docker and Terraform
Question: PGCLI - running in a Docker container
Answer: In case running pgcli  locally causes issues or you do not w...

Section: Module 1: Docker and Terraform
Question: PGCLI - running in a Docker container
Answer: In case running pgcli  locally causes issues or you do not w...

Section: Module 6: streaming with kafka
Question: How do I check compatibility of local and container Spark versions?
Answer: You can check the version of your local spark using spark-su...

Section: Module 6: streaming with kafka
Question: How do I check compatibility of local and container Spark versions?
Answer: You can check the version of your local spark using spark-su...



**Question 3:** What's the score for the top ranking result?

In [16]:
#find top record score
top_record_score =De_response["hits"]["hits"][0]["_score"]

top_record_score

75.70745

### Filtring

limit the questions to machine-learning-zoomcamp and return 3 results. 

**Question 4:** What's the 3rd question returned by the search engine?

In [20]:
ml_q = "How do I execute a command in a running docker container?"
ml_course = "machine-learning-zoomcamp"
ml_size = 3

In [21]:
#implement elastic search function
def elastic_search(query, size, course):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = e_search.search(index=index_name, body=search_query)
    response['hits']['hits']   
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs



In [22]:
ml_response = elastic_search(ml_q, ml_size, ml_course)
third_question =  ml_response[-1]
#print(f"Section: {third_question['section']}")
print(f"Question: {third_question['question']}")
#print(f"Answer: {third_question['text'][:60]}...\n")


Question: How do I debug a docker container?


### Building a prompt

**Question 5:** What's the length of the resulting prompt? (use the len function)

In [23]:
#def build_prompt(query):
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context = ""
results = elastic_search(ml_q, ml_size, ml_course)
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
prompt = prompt_template.format(question=ml_q, context=context).strip()
len(prompt)

1497

In [ ]:
! pip install tiktoken

In [24]:

import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-4o")

tokens = tokenizer.encode(prompt)

num_tokens = len(tokens)

print(f"Number of tokens: {num_tokens}")


Number of tokens: 332
